# Solving FizzBuzz with Keras

## Overview
- This notebook provides an example Keras network to solve FizzBuzz.
- This is a supervised classification problem, so we'll train our model on labeled data.

<img src="https://camo.githubusercontent.com/fedd5d66bea57a430635498de58dc7c6f064f280/68747470733a2f2f64707a6268796262327064636a2e636c6f756466726f6e742e6e65742f63686f6c6c65742f466967757265732f303166696730322e6a7067">

## What is FizzBuzz?
FizzBuzz is a common programming interview problem. Here's the setup.
- Write a program that outputs all the integers from 1 to 100.
- If the number is a multpile of 3, then output the string "Fizz"
- If the number is a multiple of 5, then output the string "Buzz"
- If the number is a multiple of 15, output the string "FizzBuzz"
- For any other number, output that, number itself.

In [1]:
import warnings
warnings.filterwarnings('ignore')

# imports
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils
from keras.models import Model

Using TensorFlow backend.


In [2]:
# Specify the number of binary digits. 
NUM_DIGITS = 10

In [3]:
## Setup the training data for 101-1024. 1024 is the highest number countable with 10 binary digits
raw_training_data = np.array(range(101, 2**NUM_DIGITS))

In [4]:
## This is a numpy array of integers.
raw_training_data[0:20]

array([101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113,
       114, 115, 116, 117, 118, 119, 120])

In [5]:
# We need to binary encode our inputs
def binary_encode(i, NUM_DIGITS):
    return np.array([i >> d & 1 for d in range(NUM_DIGITS)])

In [6]:
print(1, binary_encode(1, NUM_DIGITS))
print(2, binary_encode(2, NUM_DIGITS))
print(3, binary_encode(3, NUM_DIGITS))
print(4, binary_encode(4, NUM_DIGITS))


1 [1 0 0 0 0 0 0 0 0 0]
2 [0 1 0 0 0 0 0 0 0 0]
3 [1 1 0 0 0 0 0 0 0 0]
4 [0 0 1 0 0 0 0 0 0 0]


In [7]:
binary_encode(3, NUM_DIGITS)

array([1, 1, 0, 0, 0, 0, 0, 0, 0, 0])

In [8]:
print(256, binary_encode(256, NUM_DIGITS))
print(512, binary_encode(512, NUM_DIGITS))
print(1023, binary_encode(1023, NUM_DIGITS))

256 [0 0 0 0 0 0 0 0 1 0]
512 [0 0 0 0 0 0 0 0 0 1]
1023 [1 1 1 1 1 1 1 1 1 1]


In [9]:
encoded_training_data = [binary_encode(i, NUM_DIGITS) for i in raw_training_data]

In [10]:
x_train = np.array(encoded_training_data)
x_train[0]

array([1, 0, 1, 0, 0, 1, 1, 0, 0, 0])

In [11]:
# List out the state space. What are the different states that the data could be in:
# there's only 4 options/states in the FizzBuzz setup
# A number is a multiple of 3 and 5, only 3, only 5, or it ain't
# Now we'll need to one hot encode the training data for y
def fizz_buzz_encode(i):
    if i % 15 == 0:
        return np.array([0, 0, 0, 1]) # encoding for "fizzbuzz"
    elif i % 5 == 0: 
        return np.array([0, 0, 1, 0]) # encoding for "buzz"
    elif i % 3  == 0: 
        return np.array([0, 1, 0, 0]) # encoding for "fizz"
    else:
        return np.array([1, 0, 0, 0]) # encoding for the number output

In [12]:
print("Return only the number", fizz_buzz_encode(1))
print("Multiple of 3", fizz_buzz_encode(3))
print("Multiple of 5", fizz_buzz_encode(5))
print("Return only the number", fizz_buzz_encode(7))
print("Multiple of both 3 and 5", fizz_buzz_encode(15))

Return only the number [1 0 0 0]
Multiple of 3 [0 1 0 0]
Multiple of 5 [0 0 1 0]
Return only the number [1 0 0 0]
Multiple of both 3 and 5 [0 0 0 1]


In [13]:
# y_train is the encoded output. 
# This is our "labeled data" for supervised learning
y_train = np.array([fizz_buzz_encode(i) for i in range(101, 2 ** NUM_DIGITS)])
y_train

array([[1, 0, 0, 0],
       [0, 1, 0, 0],
       [1, 0, 0, 0],
       ...,
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [0, 1, 0, 0]])

In [14]:
# Now let's build our model, add layers, compile, and fit it!
model = Sequential()

In [15]:
## Add the model's layers. 
model.add(Dense(1000, input_dim=NUM_DIGITS, activation="relu"))
model.add(Dense(1000, activation="relu"))
model.add(Dense(4, activation="softmax"))

In [16]:
model.compile(loss='categorical_crossentropy', optimizer='adagrad', metrics=["accuracy"])

In [17]:
model.fit(x_train, y_train, nb_epoch=100, batch_size=128)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Epoch 1/100
923/923 [==============================] - 1s 850us/step - loss: 1.7365 - acc: 0.4594
Epoch 2/100
923/923 [==============================] - 0s 146us/step - loss: 1.1520 - acc: 0.5341
Epoch 3/100
923/923 [==============================] - 0s 162us/step - loss: 1.1397 - acc: 0.5341
Epoch 4/100
923/923 [==============================] - 0s 153us/step - loss: 1.1357 - acc: 0.5341
Epoch 5/100
923/923 [==============================] - 0s 154us/step - loss: 1.1256 - acc: 0.5341
Epoch 6/100
923/923 [==============================] - 0s 146us/step - loss: 1.1145 - acc: 0.5352
Epoch 7/100
923/923 [==============================] - 0s 149us/step - loss: 1.1038 - acc: 0.5330
Epoch 8/100
923/923 [==============================] - 0s 151us/step - loss: 1.0925 - acc: 0.5363
Epoch 9/100
923/923 [==============================] - 0s 155us/step - loss: 1.0736 - acc: 0.5341
Epoch 10/100
923/923 [=

923/923 [==============================] - 0s 151us/step - loss: 0.0412 - acc: 1.0000
Epoch 79/100
923/923 [==============================] - 0s 137us/step - loss: 0.0388 - acc: 1.0000
Epoch 80/100
923/923 [==============================] - 0s 210us/step - loss: 0.0380 - acc: 1.0000
Epoch 81/100
923/923 [==============================] - 0s 198us/step - loss: 0.0363 - acc: 1.0000
Epoch 82/100
923/923 [==============================] - 0s 156us/step - loss: 0.0354 - acc: 0.9989
Epoch 83/100
923/923 [==============================] - 0s 160us/step - loss: 0.0338 - acc: 1.0000
Epoch 84/100
923/923 [==============================] - 0s 168us/step - loss: 0.0341 - acc: 1.0000
Epoch 85/100
923/923 [==============================] - 0s 139us/step - loss: 0.0325 - acc: 1.0000
Epoch 86/100
923/923 [==============================] - 0s 158us/step - loss: 0.0307 - acc: 1.0000
Epoch 87/100
923/923 [==============================] - 0s 193us/step - loss: 0.0301 - acc: 1.0000
Epoch 88/100
923/923 [=

In [18]:
# convert fizzbuzz to binary
def fizz_buzz(i, prediction):
    return [str(i), "fizz", "buzz", "fizzbuzz"][prediction]


In [19]:
# Setup x_test  fizzbuzz for prime numbers from 1 to 100
numbers = np.arange(1, 101)
x_test = np.transpose(binary_encode(numbers, NUM_DIGITS))
y_test = model.predict_classes(x_test)

In [20]:
# Setup predicted output
predictions = np.vectorize(fizz_buzz)(numbers, y_test)
print (predictions)

['1' '2' 'fizz' '4' 'buzz' 'fizz' '7' '8' 'fizz' 'buzz' '11' 'fizz' '13'
 '14' 'fizzbuzz' '16' '17' 'fizz' '19' '20' 'fizz' '22' '23' 'fizz' 'buzz'
 '26' 'fizz' '28' '29' 'fizzbuzz' '31' '32' 'fizz' '34' 'buzz' 'fizz' '37'
 '38' 'fizz' 'buzz' '41' 'fizz' '43' '44' 'fizzbuzz' '46' '47' 'fizz' '49'
 'buzz' 'fizz' '52' '53' 'fizz' 'buzz' '56' 'fizz' '58' '59' 'fizzbuzz'
 '61' '62' 'fizz' '64' '65' 'fizz' '67' '68' 'fizz' 'buzz' '71' 'fizz'
 '73' '74' 'fizzbuzz' '76' '77' 'fizz' '79' '80' 'fizz' '82' '83' 'fizz'
 'buzz' '86' 'fizz' '88' '89' 'fizzbuzz' '91' '92' 'fizz' '94' 'buzz'
 'fizz' '97' '98' 'fizz' 'buzz']


In [21]:
# correct answers for fizzbuzz on 1-100, these are our actual values
answer = np.array([])
for i in numbers:
    if i % 15 == 0: 
        answer = np.append(answer, "fizzbuzz")
    elif i % 5 == 0: 
        answer = np.append(answer, "buzz")
    elif i % 3 == 0: 
        answer = np.append(answer, "fizz")
    else: answer = np.append(answer, str(i))
print (answer)

['1' '2' 'fizz' '4' 'buzz' 'fizz' '7' '8' 'fizz' 'buzz' '11' 'fizz' '13'
 '14' 'fizzbuzz' '16' '17' 'fizz' '19' 'buzz' 'fizz' '22' '23' 'fizz'
 'buzz' '26' 'fizz' '28' '29' 'fizzbuzz' '31' '32' 'fizz' '34' 'buzz'
 'fizz' '37' '38' 'fizz' 'buzz' '41' 'fizz' '43' '44' 'fizzbuzz' '46' '47'
 'fizz' '49' 'buzz' 'fizz' '52' '53' 'fizz' 'buzz' '56' 'fizz' '58' '59'
 'fizzbuzz' '61' '62' 'fizz' '64' 'buzz' 'fizz' '67' '68' 'fizz' 'buzz'
 '71' 'fizz' '73' '74' 'fizzbuzz' '76' '77' 'fizz' '79' 'buzz' 'fizz' '82'
 '83' 'fizz' 'buzz' '86' 'fizz' '88' '89' 'fizzbuzz' '91' '92' 'fizz' '94'
 'buzz' 'fizz' '97' '98' 'fizz' 'buzz']


In [22]:
# Let's evaluate the model's predictions
evaluate = np.array(answer == predictions)
print (np.count_nonzero(evaluate == True) / 100)

0.97


In [23]:
answer == predictions

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True, False,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True, False,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True, False,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True])

# Exercise
1. Reassign the NUM_DIGITS to be 5 and re-run the notebook. What do you notice? Take note of the accuracy number.
2. Reassign the NUM_DIGITS to be 11 and re-run the notebook. What do you notice? Take note of the accuracy number. 
3. Now try commenting out the last two model layers so there's only a single layer. Re-run the model to test for accuracy.
4. Un-comment the last two model layers and set all their their activation parameters to "sigmoid" and re-run.
5. Explore https://keras.io/activations/ to see what other activation functions are available. Experiment with some and see how they perform. This is an example of hyperparameter tuning.